# Resources
- https://www.vaksincovid.gov.my/json/ppv.json

In [56]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import json
from collections import OrderedDict
from urllib.request import Request, urlopen
import pandas as pd
import re
import csv

## Senarai Pusat Pemberian Vaksin
https://www.vaksincovid.gov.my/ppv/

In [28]:
# url = r"https://www.vaksincovid.gov.my/json/ppv.json"
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0',
#     "Referer":'https://www.vaksincovid.gov.my/ppv/',
#     'Accept'	:'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
#     # Referer: https://www.vaksincovid.gov.my/ppv/
#     # Accept	text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8
#     }
# session = requests.Session()
# response = session.get(url=url, timeout=10, headers=headers)
# response.raise_for_status()
# raw_data = response.json()
# print(raw_data)

In [32]:
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
str_data = urlopen(req).read().decode(encoding='utf-8-sig')
raw_data = json.loads(str_data)
print(json.dumps(raw_data, indent=2))

{
  "data": [
    {
      "st": "Negeri Sembilan",
      "dist": "Seremban",
      "ppvc": "AMC Medical Clinic",
      "cd": "negerisembilan",
      "lat": "2.69262",
      "lon": "101.90478",
      "linky": "https://goo.gl/maps/ZMjKwHU3YBZRvt6i9"
    },
    {
      "st": "Sarawak",
      "dist": "Bintulu",
      "ppvc": "Auditorium BDA, Wisma BDA",
      "cd": "sarawak",
      "lat": "",
      "lon": "",
      "linky": "https://goo.gl/maps/bPJkCiKm4YwE6LMb8"
    },
    {
      "st": "Sarawak",
      "dist": "Bintulu",
      "ppvc": "Auditorium BDA, Wisma Bintulu",
      "cd": "sarawak",
      "lat": "3.21315",
      "lon": "113.06177",
      "linky": ""
    },
    {
      "st": "Johor",
      "dist": "Johor Bahru",
      "ppvc": "Austin International Convention Centre - AICC",
      "cd": "johor",
      "lat": "",
      "lon": "",
      "linky": "https://goo.gl/maps/BaLoiT9hUm6bjaV5A"
    },
    {
      "st": "W.P. Kuala Lumpur",
      "dist": "Kuala Lumpur",
      "ppvc": "Axiata Are

In [30]:
response = requests.get(r"https://goo.gl/maps/y1kMiojDAQoRZmKo9")
if response.history:
    print("Request was redirected")
    for resp in response.history:
        print(resp.status_code, resp.url)
    print("Final destination:")
    print(response.status_code, response.url)
else:
    print("Request was not redirected")

Request was redirected
302 https://goo.gl/maps/y1kMiojDAQoRZmKo9
Final destination:
200 https://www.google.com/maps/place/Yayasan+Toh+Puan+Zurina/@2.2252086,102.2560359,17.75z/data=!4m5!3m4!1s0x31d1efd8dfd201f1:0x66962b307625e6a!8m2!3d2.2256667!4d102.2572954?shorturl=1


In [46]:
COOR_PT = re.compile(r"@([\d]+[.][\d]+),([\d]+[.][\d]+),")

def clean_details(details):
    data = OrderedDict()
    data["Name"]=""
    data["Clinic"]=0
    data["Hospital"]=0
    data["ST"]=0
    data["VAC"]=1
    data["Latitude"]=""
    data["Longitude"]=""
    data["Address"]=""
    data["Contact"]=""
    data["State"]=""
    
    data["Name"] = details["ppvc"].strip()
    data["State"] = details["st"].strip()

    data["Address"] = details["dist"].strip()
    data["Contact"]= ""
    
    if len(details["lat"]) > 0:
        data["Latitude"] = details["lat"]
        data["Longitude"] = details["lon"]
    elif len(details["linky"]) > 0:
        try:
            response = requests.get(url=details["linky"], timeout=10)
            response.raise_for_status()
            result = COOR_PT.findall(response.url)
            if len(result) > 0:
                data["Latitude"] = result[0][0]
                data["Longitude"] = result[0][1]
            else:
                print("{0}: {1}".format(data["Name"], "URL error"))
        except Exception as e:
            print("{0}: {1}".format(data["Name"], e))

    return data

In [49]:
data_ls = []
for row in raw_data["data"]:
    data = clean_details(row)
    data_ls.append(data)

Borneo Convention Centre Kuching (BCCK): URL error
Dewan LPK, Kelab Sukan Dan Rekreasi Pelabuhan Klang: URL error
Equatorial Hotel Penang: URL error
Hospital Pengajar UPM: URL error
Ideal Convention Centre (IDCC): URL error
Klinik Johor 14 Jam: URL error
Klinik Kesihatan Bingkor: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/maps/place/Health%2BClinic%2BBingkor/%405.4052903,116.1881577,867m/data%3D!3m2!1e3!4b1!4m5!3m4!1s0x323cedaf112a2faf:0x550f7680a47064ea!8m2!3d5.4053887!4d116.1903777%3Fshorturl%3D1&q=EgS0S4jkGKberIkGIhBSaASTvZCpBEobk1_vSUcwMgFy
Klinik Kesihatan Bukit Belacon: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/maps/place/Klinik%2BKesihatan%2BBukit%2BBelacon/%405.102023,118.2254755,867m/data%3D!3m2!1e3!4b1!4m5!3m4!1s0x323f0bb62fd55181:0xb6bac3f746dd6685!8m2!3d5.102023!4d118.2276642%3Fshorturl%3D1&q=EgS0S4jkGKferIkGIhAuDql4SfgzEZ48pRX7YWy1MgFy


In [53]:
df_1 = pd.DataFrame.from_dict(data_ls)
df_1.head(n=10)

,Name,Clinic,Hospital,ST,VAC,Latitude,Longitude,Address,Contact,State
0,AMC Medical Clinic,0,0,0,1,2.69262,101.90478,Seremban,,Negeri Sembilan
1,"Auditorium BDA, Wisma BDA",0,0,0,1,3.2081999,113.0485959,Bintulu,,Sarawak
2,"Auditorium BDA, Wisma Bintulu",0,0,0,1,3.21315,113.06177,Bintulu,,Sarawak
3,Austin International Convention Centre - AICC,0,0,0,1,1.5610537,103.7736871,Johor Bahru,,Johor
4,Axiata Arena Bukit Jalil,0,0,0,1,3.0538134,101.691247,Kuala Lumpur,,W.P. Kuala Lumpur
5,B Braun Medical Industries Sdn Bhd,0,0,0,1,3.1466205,101.6150604,Barat Daya,,Pulau Pinang
6,Balai Raya Felda Chiku 5,0,0,0,1,4.987999,102.202632,Gua Musang,,Kelantan
7,Balai Raya Sebauh,0,0,0,1,3.11774,113.26414,Sebauh,,Sarawak
8,Bangi Avenue Convention Center [BACC],0,0,0,1,2.9006795,101.7890557,Ulu Langat,,Selangor
9,"Bangi Convention Centre (BCC), Seksyen 14 BBB",0,0,0,1,2.9258159,101.7613474,Ulu Langat,,Selangor


In [58]:
df_1.to_csv(
    "CSV/VAC.csv",
    quoting=csv.QUOTE_NONNUMERIC,
    escapechar="\\",
    doublequote=False,
    index=False)